In [1]:
#Import all used libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from scipy.stats import spearmanr
import yfinance as yf
import datetime
import time
import requests
import io
from fbprophet import Prophet
import holidays
from IPython.display import display_html

Importing plotly failed. Interactive plots will not work.


# Stock data acquisition

In [2]:
#Set dates
start = datetime.datetime(2017,4,3)
end = datetime.datetime(2020,5,31)
url="https://pkgstore.datahub.io/core/nasdaq-listings/nasdaq-listed_csv/data/7665719fb51081ba0bd834fde71ce822/nasdaq-listed_csv.csv"
s = requests.get(url).content
companies = pd.read_csv(io.StringIO(s.decode('utf-8')))
Symbols=['AAL', 'DAL', 'LUV', 'UAL']
# create empty dataframe
stock_final = pd.DataFrame()

# iterate over each symbol
for i in Symbols:  
    
    # print the symbol which is being downloaded
    print( str(Symbols.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['Name']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None
#Order stocks by name
stock_final=stock_final.reset_index()
stock_final=stock_final.set_index("Name")

#Create 4 csv files to store stocks 
aaStock=stock_final.loc['AAL']
aaStock.to_csv(path_or_buf=r'AAL.csv', index=False)
uStock=stock_final.loc['UAL']
uStock.to_csv(path_or_buf=r'UAL.csv', index=False)
dStock=stock_final.loc['DAL']
dStock.to_csv(path_or_buf=r'DAL.csv', index=False)
sStock=stock_final.loc['LUV']
sStock.to_csv(path_or_buf=r'LUV.csv', index=False)

0 : AAL,1 : DAL,2 : LUV,3 : UAL,

# Fuel data acquisition

In [3]:
#HTML table obtained from webpage
HTMLstring=""""
<table class="tblData" cellspacing="0" cellpadding="3" align="Center" rules="all" border="1" id="gvPrices" style="border-collapse:collapse;">
				<tbody><tr style="background-color:#E5ECF9;">
					<th scope="col">Month</th><th scope="col">Price</th><th scope="col">Change</th>
				</tr><tr align="right">
					<td>Dec 2015</td><td>1.08</td><td>-</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Jan 2016</td><td>.93</td><td>-14.05 %</td>
				</tr><tr align="right">
					<td>Feb 2016</td><td>.97</td><td>4.62 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Mar 2016</td><td>1.07</td><td>9.87 %</td>
				</tr><tr align="right">
					<td>Apr 2016</td><td>1.15</td><td>7.30 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>May 2016</td><td>1.30</td><td>13.25 %</td>
				</tr><tr align="right">
					<td>Jun 2016</td><td>1.38</td><td>6.39 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Jul 2016</td><td>1.27</td><td>-7.96 %</td>
				</tr><tr align="right">
					<td>Aug 2016</td><td>1.30</td><td>1.81 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Sep 2016</td><td>1.32</td><td>1.85 %</td>
				</tr><tr align="right">
					<td>Oct 2016</td><td>1.46</td><td>10.46 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Nov 2016</td><td>1.36</td><td>-6.93 %</td>
				</tr><tr align="right">
					<td>Dec 2016</td><td>1.49</td><td>9.96 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Jan 2017</td><td>1.51</td><td>1.54 %</td>
				</tr><tr align="right">
					<td>Feb 2017</td><td>1.55</td><td>2.18 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Mar 2017</td><td>1.45</td><td>-6.59 %</td>
				</tr><tr align="right">
					<td>Apr 2017</td><td>1.51</td><td>4.50 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>May 2017</td><td>1.41</td><td>-6.49 %</td>
				</tr><tr align="right">
					<td>Jun 2017</td><td>1.30</td><td>-8.29 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Jul 2017</td><td>1.42</td><td>9.42 %</td>
				</tr><tr align="right">
					<td>Aug 2017</td><td>1.56</td><td>10.16 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Sep 2017</td><td>1.80</td><td>15.31 %</td>
				</tr><tr align="right">
					<td>Oct 2017</td><td>1.66</td><td>-7.83 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Nov 2017</td><td>1.76</td><td>6.09 %</td>
				</tr><tr align="right">
					<td>Dec 2017</td><td>1.82</td><td>3.24 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Jan 2018</td><td>1.95</td><td>7.43 %</td>
				</tr><tr align="right">
					<td>Feb 2018</td><td>1.85</td><td>-5.28 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Mar 2018</td><td>1.86</td><td>0.49 %</td>
				</tr><tr align="right">
					<td>Apr 2018</td><td>2.02</td><td>8.45 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>May 2018</td><td>2.16</td><td>6.95 %</td>
				</tr><tr align="right">
					<td>Jun 2018</td><td>2.09</td><td>-3.02 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Jul 2018</td><td>2.10</td><td>0.38 %</td>
				</tr><tr align="right">
					<td>Aug 2018</td><td>2.12</td><td>0.95 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Sep 2018</td><td>2.19</td><td>3.35 %</td>
				</tr><tr align="right">
					<td>Oct 2018</td><td>2.25</td><td>2.74 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Nov 2018</td><td>1.95</td><td>-13.52 %</td>
				</tr><tr align="right">
					<td>Dec 2018</td><td>1.70</td><td>-12.80 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Jan 2019</td><td>1.78</td><td>5.19 %</td>
				</tr><tr align="right">
					<td>Feb 2019</td><td>1.91</td><td>7.29 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Mar 2019</td><td>1.90</td><td>-0.63 %</td>
				</tr><tr align="right">
					<td>Apr 2019</td><td>1.98</td><td>3.94 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>May 2019</td><td>1.97</td><td>-0.25 %</td>
				</tr><tr align="right">
					<td>Jun 2019</td><td>1.82</td><td>-7.86 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Jul 2019</td><td>1.91</td><td>5.28 %</td>
				</tr><tr align="right">
					<td>Aug 2019</td><td>1.80</td><td>-5.91 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Sep 2019</td><td>1.88</td><td>4.28 %</td>
				</tr><tr align="right">
					<td>Oct 2019</td><td>1.86</td><td>-0.85 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Nov 2019</td><td>1.82</td><td>-1.99 %</td>
				</tr><tr align="right">
					<td>Dec 2019</td><td>1.89</td><td>3.78 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Jan 2020</td><td>1.78</td><td>-6.13 %</td>
				</tr><tr align="right">
					<td>Feb 2020</td><td>1.51</td><td>-15.08 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Mar 2020</td><td>.95</td><td>-36.85 %</td>
				</tr><tr align="right">
					<td>Apr 2020</td><td>.61</td><td>-36.41 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>May 2020</td><td>.69</td><td>13.20 %</td>
				</tr><tr align="right">
					<td>Jun 2020</td><td>.98</td><td>43.29 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Jul 2020</td><td>1.08</td><td>10.27 %</td>
				</tr><tr align="right">
					<td>Aug 2020</td><td>1.11</td><td>2.58 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Sep 2020</td><td>1.01</td><td>-9.53 %</td>
				</tr><tr align="right">
					<td>Oct 2020</td><td>1.05</td><td>4.08 %</td>
				</tr><tr align="right" style="background-color:#EFEFEF;">
					<td>Nov 2020</td><td>1.13</td><td>8.21 %</td>
				</tr>
			</tbody></table>"""

In [4]:
dfs = pd.read_html(HTMLstring)
df=dfs[0]
fuel_p=df[16:55]
fuel_p=fuel_p.reset_index()
fuel_p.to_csv(path_or_buf=r'Fuel_prices.csv', index=False)

# Passenger data acquisition


Passengers before March 2020

In [5]:
HTMLstring=""""
<table cellspacing="1" cellpadding="4" rules="all" border="1" id="GridView1" style="width:750px;">
		<tbody><tr class="libraryTHY2_Center" style="color:White;background-color:#5D95C9;">
			<th scope="col">Year</th><th scope="col">Month</th><th scope="col">DOMESTIC</th><th scope="col">INTERNATIONAL</th><th scope="col">TOTAL</th>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2002</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">48,054,917</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9,578,435</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,633,352</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2002</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">44,850,246</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9,016,535</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,866,781</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2002</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">49,684,353</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,038,794</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,723,147</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2002</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>551,899,643</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>118,704,850</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>670,604,493</b></td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2003</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">43,032,450</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9,726,436</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,758,886</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2003</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">41,166,780</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8,283,372</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">49,450,152</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2003</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">49,992,700</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9,538,653</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,531,353</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2003</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,033,260</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8,309,305</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,342,565</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2003</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">49,152,352</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8,801,873</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,954,225</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2003</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,209,516</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,347,900</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">62,557,416</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2003</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,810,773</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,705,206</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">67,515,979</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2003</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,920,973</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,799,672</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">65,720,645</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2003</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">44,213,408</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9,454,647</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,668,055</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2003</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">49,944,935</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9,608,358</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,553,293</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2003</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,059,495</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9,481,886</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">56,541,381</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2003</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">49,757,124</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,512,547</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">60,269,671</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2003</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>583,293,766</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>117,569,855</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>700,863,621</b></td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">43,815,481</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,252,443</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,067,924</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">45,306,644</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9,310,317</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,616,961</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,147,227</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,976,440</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">65,123,667</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,253,194</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,802,022</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,055,216</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,030,873</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,971,254</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,002,127</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">56,959,142</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,159,514</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">69,118,656</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,614,287</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,502,913</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">73,117,200</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,380,873</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,203,566</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">70,584,439</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,671,785</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,570,682</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,242,467</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,167,489</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,772,177</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,939,666</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">51,782,564</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,174,320</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,956,884</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,640,057</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,244,427</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,884,484</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2004</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>629,769,616</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>133,940,075</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>763,709,691</b></td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2005</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,977,657</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,387,275</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,364,932</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2005</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,074,882</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9,945,067</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,019,949</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2005</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,838,975</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,479,643</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">71,318,618</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2005</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,908,859</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,521,174</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">66,430,033</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2005</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,323,876</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,164,600</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">69,488,476</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2005</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,724,061</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,140,061</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,864,122</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2005</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">62,396,446</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,421,209</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">76,817,655</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2005</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,110,633</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,703,334</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,813,967</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2005</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">50,600,325</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,325,800</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,926,125</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2005</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,738,093</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,107,586</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,845,679</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2005</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,766,404</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,580,099</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,346,503</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2005</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,801,276</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,812,574</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,613,850</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2005</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>657,261,487</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>143,588,422</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>800,849,909</b></td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2006</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">48,886,043</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,724,098</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">60,610,141</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2006</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,348,142</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,167,275</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,515,417</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2006</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,286,011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,707,241</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">70,993,252</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2006</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,828,555</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,619,137</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">68,447,692</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2006</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,145,193</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,610,793</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">69,755,986</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2006</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,297,121</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,690,835</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,987,956</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2006</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">60,838,606</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,912,621</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">75,751,227</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2006</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,303,233</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,171,796</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,475,029</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2006</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">49,949,551</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,010,922</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,960,473</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2006</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,088,986</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,517,228</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">66,606,214</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2006</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,852,209</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,222,687</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">65,074,896</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2006</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,538,970</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,385,958</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">65,924,928</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2006</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>658,362,620</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>149,740,591</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>808,103,211</b></td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2007</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">50,022,168</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,072,442</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">62,094,610</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2007</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,766,421</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,591,580</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,358,001</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2007</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,244,232</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,247,375</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,491,607</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2007</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,398,267</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,795,517</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">70,193,784</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2007</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,285,571</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,972,589</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,258,160</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2007</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,493,934</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,170,877</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">75,664,811</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2007</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,457,403</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,470,687</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">78,928,090</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2007</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">62,660,179</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,131,296</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">77,791,475</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2007</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,310,402</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,332,434</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,642,836</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2007</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,208,618</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,392,938</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">69,601,556</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2007</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,047,116</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,023,481</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">67,070,597</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2007</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,291,139</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,123,756</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">66,414,895</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2007</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>679,185,450</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>156,324,972</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>835,510,422</b></td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2008</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">50,245,100</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,794,514</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,039,614</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2008</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">50,118,363</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,677,400</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,795,763</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2008</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,213,077</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,221,964</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">73,435,041</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2008</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,611,053</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,983,854</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">68,594,907</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2008</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,047,400</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,628,619</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">71,676,019</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2008</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,559,171</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,447,504</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">74,006,675</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2008</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,401,480</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,488,628</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">76,890,108</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2008</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,966,473</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,369,492</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">74,335,965</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2008</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,680,101</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,759,694</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,439,795</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2008</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,956,023</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,952,515</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,908,538</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2008</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,658,141</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,288,311</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,946,452</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2008</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">50,253,800</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,499,216</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">62,753,016</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2008</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>651,710,182</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>158,111,711</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>809,821,893</b></td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2009</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">44,825,056</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,075,267</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">56,900,323</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2009</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">43,680,328</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,412,676</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,093,004</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2009</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,534,891</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,508,793</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">66,043,684</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2009</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,247,781</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,629,805</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,877,586</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2009</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,951,246</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,806,548</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,757,794</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2009</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,898,027</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,199,636</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">69,097,663</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2009</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,468,957</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,953,615</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">74,422,572</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2009</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">56,568,180</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,844,821</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">71,413,001</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2009</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">48,235,025</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,659,775</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,894,800</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2009</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,279,405</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,905,529</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,184,934</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2009</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">48,584,832</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,093,714</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,678,546</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2009</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">49,793,527</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,659,154</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">62,452,681</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2009</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>618,067,255</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>149,749,333</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>767,816,588</b></td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2010</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">45,501,620</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,393,439</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,895,059</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2010</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">42,440,614</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,694,165</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,134,779</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2010</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,424,077</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,279,320</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">67,703,397</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2010</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,498,074</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,398,700</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,896,774</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2010</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,842,422</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,380,664</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">67,223,086</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2010</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">56,689,853</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,406,776</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">71,096,629</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2010</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,142,493</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,026,852</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">75,169,345</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2010</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,370,748</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,317,926</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,688,674</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2010</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">50,304,708</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,599,149</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">62,903,857</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2010</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,826,050</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,874,707</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">67,700,757</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2010</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">51,420,519</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,651,314</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,071,833</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2010</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">51,076,415</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,917,451</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,993,866</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2010</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>629,537,593</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>157,940,463</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>787,478,056</b></td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">46,305,000</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,668,763</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,973,763</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">43,657,818</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10,944,610</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,602,428</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,649,428</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,473,705</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">69,123,133</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,858,479</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,537,152</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">66,395,631</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,982,234</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,935,132</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">69,917,366</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,778,699</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,814,672</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,593,371</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">60,310,568</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,623,674</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">76,934,242</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,403,835</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,753,438</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">73,157,273</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">50,982,170</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,220,884</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,203,054</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,124,363</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,199,913</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">67,324,276</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">51,660,083</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,156,133</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,816,216</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2011</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">51,534,990</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,559,050</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">65,094,040</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2011</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>638,247,667</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>163,887,126</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>802,134,793</b></td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2012</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,080,840</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,206,801</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">60,287,641</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2012</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">46,405,385</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11,732,944</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,138,329</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2012</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">56,200,391</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,387,677</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">70,588,068</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2012</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,691,570</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,995,377</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">67,686,947</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2012</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,749,349</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,291,285</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">70,040,634</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2012</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,903,293</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,582,335</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">73,485,628</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2012</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,688,267</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,969,590</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">76,657,857</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2012</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,646,304</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,616,643</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">75,262,947</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2012</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">50,134,504</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,791,714</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,926,218</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2012</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,779,646</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,397,785</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">67,177,431</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2012</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">51,844,482</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,763,847</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,608,329</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2012</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">51,165,451</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,097,885</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">65,263,336</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2012</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>642,289,482</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>170,833,883</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>813,123,365</b></td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2013</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,816,859</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,768,819</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,585,678</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2013</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">45,740,158</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,037,584</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,777,742</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2013</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">56,564,538</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,188,333</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">71,752,871</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2013</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">53,228,575</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,301,619</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">67,530,194</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2013</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">56,562,929</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,353,665</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">71,916,594</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2013</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,990,789</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,403,199</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">74,393,988</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2013</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,310,478</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,748,869</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">77,059,347</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2013</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,115,405</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,666,442</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">75,781,847</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2013</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">50,766,098</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,368,204</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">65,134,302</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2013</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,706,246</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,341,721</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">69,047,967</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2013</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">50,542,543</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,260,466</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,803,009</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2013</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,332,936</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,205,588</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">69,538,524</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2013</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>645,677,554</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>179,644,509</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>825,322,063</b></td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2014</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,957,371</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,692,606</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">62,649,977</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2014</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">45,513,184</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12,482,600</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,995,784</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2014</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,762,170</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,604,456</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">73,366,626</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2014</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,247,482</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,471,267</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">70,718,749</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2014</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,888,637</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,338,378</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">74,227,015</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2014</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,262,095</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,361,717</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">76,623,812</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2014</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,757,698</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,784,523</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">80,542,221</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2014</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,758,814</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,784,355</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">78,543,169</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2014</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,529,472</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,985,203</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">67,514,675</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2014</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,084,923</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,981,228</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,066,151</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2014</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,578,037</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,730,768</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">66,308,805</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2014</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,487,072</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,805,756</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">71,292,828</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2014</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>662,826,955</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>189,022,857</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>851,849,812</b></td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2015</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">49,727,992</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,467,163</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">65,195,155</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2015</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">47,164,898</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">13,325,293</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">60,490,191</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2015</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,557,626</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,342,061</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">75,899,687</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2015</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,698,342</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,025,595</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">73,723,937</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2015</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">60,252,607</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,115,194</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">77,367,801</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2015</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,811,276</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,203,199</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">80,014,475</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2015</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">65,129,348</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">20,215,002</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">85,344,350</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2015</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">62,770,806</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">19,914,264</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">82,685,070</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2015</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,989,599</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,111,956</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,101,555</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2015</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">60,892,617</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,188,843</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">77,081,460</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2015</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,150,828</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,846,611</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">71,997,439</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2015</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,870,955</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,859,827</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">74,730,782</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2015</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>696,016,894</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>200,615,008</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>896,631,902</b></td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2016</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">52,474,245</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,492,079</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">68,966,324</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2016</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">51,111,616</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,421,545</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">65,533,161</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2016</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,593,610</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,382,692</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">78,976,302</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2016</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,894,278</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,596,730</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">75,491,008</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2016</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">62,752,723</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,868,001</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">80,620,724</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2016</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,757,943</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">19,407,251</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">84,165,194</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2016</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">66,135,126</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">21,435,123</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">87,570,249</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2016</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,498,615</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">20,654,207</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">84,152,822</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2016</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,620,581</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,161,889</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">75,782,470</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2016</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,709,421</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,905,446</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">78,614,867</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2016</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,269,837</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,653,035</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">74,922,872</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2016</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,178,833</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,014,473</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">77,193,306</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2016</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>719,996,828</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>211,992,471</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>931,989,299</b></td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2017</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,109,329</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,488,782</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">71,598,111</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2017</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">51,078,172</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">14,709,988</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">65,788,160</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2017</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,963,808</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,910,122</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">81,873,930</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2017</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,099,004</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,686,523</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">79,785,527</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2017</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,442,977</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,865,393</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">83,308,370</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2017</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">66,746,463</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">20,676,407</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">87,422,870</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2017</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">68,575,481</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">22,690,243</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">91,265,724</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2017</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">66,664,019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">22,013,119</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">88,677,138</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2017</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">57,144,508</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,438,756</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">74,583,264</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2017</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,625,405</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,604,236</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">82,229,641</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2017</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,911,765</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,476,642</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">78,388,407</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2017</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,374,167</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,469,636</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">79,843,803</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2017</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>741,735,098</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>223,029,847</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>964,764,945</b></td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2018</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,831,159</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,897,787</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">73,728,946</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2018</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">54,078,219</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,720,901</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">69,799,120</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2018</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">66,644,135</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">19,716,774</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">86,360,909</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2018</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,557,780</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">19,111,955</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">83,669,735</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2018</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">67,846,119</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">20,126,585</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">87,972,704</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2018</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">70,279,158</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">22,046,732</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">92,325,890</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2018</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,540,389</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">23,754,573</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">96,294,962</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2018</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">70,338,563</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">22,909,627</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">93,248,190</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2018</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">60,468,575</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,051,374</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">78,519,949</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2018</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">67,082,086</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,830,854</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">85,912,940</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2018</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,660,022</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,552,376</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">82,212,398</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2018</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,646,582</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">19,520,179</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">83,166,761</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2018</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>777,972,787</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>235,239,717</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>1,013,212,504</b></td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">58,034,262</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,690,817</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">76,725,079</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">55,679,481</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,280,536</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">71,960,017</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">70,234,684</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">20,295,947</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">90,530,631</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">66,938,654</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">20,019,304</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">86,957,958</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">71,365,660</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">20,995,113</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">92,360,773</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,790,418</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">22,747,453</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">95,537,871</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">75,281,916</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">23,907,859</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">99,189,775</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">72,729,540</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">23,239,338</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">95,968,878</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">63,991,365</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,842,310</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">82,833,675</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">69,936,836</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,973,444</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">88,910,280</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">11</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">64,827,107</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,575,113</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">82,402,220</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2019</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">12</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">69,737,246</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">19,875,311</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">89,612,557</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2019</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>811,547,169</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>241,442,545</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>1,052,989,714</b></td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2020</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">61,627,591</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">18,946,132</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">80,573,723</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2020</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">59,861,339</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">15,823,231</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">75,684,570</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2020</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">34,417,634</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8,924,129</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">43,341,763</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2020</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">4</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2,879,554</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">396,394</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">3,275,948</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2020</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">5</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8,238,727</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">369,317</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8,608,044</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2020</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">6</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">16,207,526</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">801,741</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">17,009,267</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2020</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">7</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">22,878,515</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">1,847,382</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">24,725,897</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2020</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">8</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">23,913,387</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2,267,701</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">26,181,088</td>
		</tr><tr class="dataTDRight" style="background-color:#EFEFEF;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2020</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">9</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">23,853,978</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2,237,363</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">26,091,341</td>
		</tr><tr class="dataTDRight">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2020</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">10</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">28,022,821</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">2,907,561</td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;">30,930,382</td>
		</tr><tr class="dataTDRight" style="background-color:LightYellow;">
			<td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>2020</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>TOTAL</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>281,901,072</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>54,520,951</b></td><td style="font-family: Verdana, Geneva, Arial, Helvetica, sans-serif;"><b>336,422,023</b></td>
		</tr>
	</tbody></table>"""

In [6]:
dfs = pd.read_html(HTMLstring)
df=dfs[0]
df=df.set_index("Month")
df=df.drop("TOTAL")
df=df.reset_index()
Pass=df[174:209]
Pass.to_csv(path_or_buf=r'pass_prev.csv', index=False)

Passengers from May 2019

In [7]:
HTMLstring="""
<table class="views-table views-view-table cols-4 sticky-enabled sticky-table">
        <thead>
      <tr>
                                                  <th class="views-align-center views-field views-field-field-today-date views-align-center" id="view-field-today-date-table-column" scope="col">Date</th>
                                                  <th class="views-align-center views-field views-field-field-2021-throughput views-align-center" id="view-field-2021-throughput-table-column" scope="col">2021 Traveler Throughput </th>
                                                  <th class="views-align-center views-field views-field-field-2020-throughput views-align-center" id="view-field-2020-throughput-table-column" scope="col">2020 Traveler Throughput </th>
                                                  <th class="views-align-center views-field views-field-field-2019-throughput views-align-center" id="view-field-2019-throughput-table-column" scope="col">2019 Traveler Throughput </th>
              </tr>
    </thead>
    <tbody>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/20/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">542,338          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,801,444          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,776,235          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/19/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">560,190          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,870,459          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,600,698          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/18/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">878,048          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,298,616          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,271,398          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/17/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">810,654          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,000,260          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,139,242          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/16/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">690,438          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,781,893          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,786,012          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/15/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">903,039          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,347,075          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,605,758          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/14/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">803,688          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,242,656          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,886,642          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/13/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">567,401          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,876,782          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,970,450          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/12/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">520,117          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,691,205          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,604,862          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/11/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">708,177          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,992,453          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,959,788          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/10/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">886,536          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,183,734          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,955,200          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/9/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">709,444          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,687,974          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,739,642          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/8/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">772,471          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,072,543          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,733,739          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/7/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">771,734          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,034,472          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,044,043          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/6/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">665,855          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,815,040          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,229,391          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/5/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">766,594          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,806,480          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,975,947          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/4/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">1,080,346          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,210,542          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,150,571          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/3/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">1,327,289          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,422,272          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,202,111          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/2/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">1,192,881          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,178,656          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,345,103          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">1/1/2021          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">805,990          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,311,732          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,126,398          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/31/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">874,406          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,392,331          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/30/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,163,696          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,133,253          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/29/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,019,347          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,009,880          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/28/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,111,751          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,500,396          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/27/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,284,599          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,575,985          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/26/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,128,773          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,470,786          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/25/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">616,469          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,582,580          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/24/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">846,520          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,552,194          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/23/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,191,123          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,937,235          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/22/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">992,167          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,981,433          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/21/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">954,782          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,490,503          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/20/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,064,619          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,519,399          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/19/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,073,563          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,487,987          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/18/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,066,747          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,608,088          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/17/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">846,934          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,471,408          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/16/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">641,966          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,234,374          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/15/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">552,024          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,009,112          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/14/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">752,451          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,250,386          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/13/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">865,014          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,300,248          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/12/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">662,380          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,893,871          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/11/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">787,489          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,388,029          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/10/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">754,307          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,362,310          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/9/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">564,372          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,020,488          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/8/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">501,513          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,897,051          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/7/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">703,546          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,226,290          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/6/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">837,137          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,292,079          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/5/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">629,430          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,755,801          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/4/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">753,951          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,278,205          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/3/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">738,050          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,262,878          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/2/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">632,356          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,054,380          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">12/1/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">780,283          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,280,403          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/30/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">981,912          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,591,470          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/29/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,176,091          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,882,915          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/28/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">964,630          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,648,268          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/27/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">820,399          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,968,137          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/26/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">560,902          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,591,158          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/25/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,070,967          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,624,250          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/24/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">912,090          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,435,170          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/23/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">917,354          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,254,188          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/22/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,047,934          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,321,546          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/21/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">984,369          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,194,291          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/20/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,019,836          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,550,459          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/19/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">907,332          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,428,095          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/18/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">703,135          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,071,631          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/17/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">611,497          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,900,895          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/16/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">883,157          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,298,856          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/15/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">978,297          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,396,681          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/14/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">697,360          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,807,230          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/13/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">881,579          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,437,211          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/12/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">866,679          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,364,920          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/11/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">674,633          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,072,207          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/10/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">596,475          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,150,003          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/9/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">836,600          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,465,392          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/8/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">973,020          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,356,349          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/7/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">689,951          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,908,805          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/6/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">895,091          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,544,350          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/5/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">867,105          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,507,365          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/4/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">636,533          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,147,882          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/3/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">575,829          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,005,101          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/2/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">846,138          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,403,304          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">11/1/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">936,092          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,459,525          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/31/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">618,476          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,836,781          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/30/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">892,712          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,319,906          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/29/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">873,636          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,047,910          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/28/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">666,957          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,066,516          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/27/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">648,517          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,910,506          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/26/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">898,735          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,347,017          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/25/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">983,745          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,478,287          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/24/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">755,287          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,931,971          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/23/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">958,437          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,594,337          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/22/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">934,386          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,541,581          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/21/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">694,150          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,245,199          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/20/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">662,484          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,126,637          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/19/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">921,031          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,514,673          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/18/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,031,505          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,606,266          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/17/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">788,743          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,049,855          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/16/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">973,046          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,637,667          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/15/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">950,024          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,581,007          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/14/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">717,940          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,317,763          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/13/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">680,894          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,313,632          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/12/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">958,440          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,616,771          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/11/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">984,234          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,555,333          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/10/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">769,868          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,074,718          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/9/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">968,545          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,688,032          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/8/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">936,915          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,605,291          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/7/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">668,519          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,215,233          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/6/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">590,766          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,035,628          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/5/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">816,838          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,400,153          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/4/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">900,911          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,542,118          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/3/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">677,661          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,921,185          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/2/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">857,186          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,526,835          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">10/1/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">855,908          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,447,687          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/30/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">634,046          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,082,179          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/29/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">568,688          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,998,980          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/28/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">797,699          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,368,818          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/27/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">873,038          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,452,596          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/26/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">659,350          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,966,234          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/25/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">826,329          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,547,611          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/24/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">826,316          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,510,926          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/23/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">608,726          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,188,236          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/22/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">549,741          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,033,490          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/21/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">769,936          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,431,388          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/20/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">847,968          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,517,826          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/19/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">638,575          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,938,402          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/18/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">812,214          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,571,924          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/17/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">784,746          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,455,410          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/16/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">577,847          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,146,857          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/15/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">522,383          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,013,050          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/14/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">729,558          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,405,832          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/13/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">809,850          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,485,134          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/12/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">613,703          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,879,822          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/11/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">731,353          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,484,025          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/10/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">755,051          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,449,302          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/9/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">616,923          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,005,867          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/8/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">704,075          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,943,379          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/7/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">935,308          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,292,985          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/6/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">689,630          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,370,003          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/5/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">664,640          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,755,502          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/4/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">968,673          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,198,828          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/3/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">877,698          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,109,858          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/2/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">578,131          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,889,044          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">9/1/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">516,068          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,037,750          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/31/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">711,178          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,278,159          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/30/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">807,695          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,887,845          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/29/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">591,734          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,954,902          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/28/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">738,873          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,658,558          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/27/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">721,060          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,561,109          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/26/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">540,043          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,188,688          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/25/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">523,186          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,015,088          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/24/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">726,788          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,358,007          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/23/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">841,806          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,493,162          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/22/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">625,822          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,039,233          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/21/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">764,468          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,559,244          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/20/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">772,380          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,533,184          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/19/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">586,718          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,306,838          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/18/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">565,946          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,247,446          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/17/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">773,319          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,576,965          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/16/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">862,949          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,584,444          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/15/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">689,895          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,171,962          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/14/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">783,744          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,627,564          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/13/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">761,821          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,602,446          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/12/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">590,749          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,391,906          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/11/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">559,420          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,306,829          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/10/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">761,861          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,567,986          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/9/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">831,789          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,647,897          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/8/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">683,212          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,290,340          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/7/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">762,547          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,725,000          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/6/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">743,599          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,707,986          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/5/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">595,739          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,430,094          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/4/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">543,601          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,387,115          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/3/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">737,235          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,619,406          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/2/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">799,861          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,688,640          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">8/1/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">709,033          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,367,967          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/31/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">767,320          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,730,936          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/30/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">718,310          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,742,882          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/29/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">573,200          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,542,365          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/28/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">536,756          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,438,967          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/27/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">700,043          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,613,346          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/26/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">751,205          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,700,723          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/25/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">649,027          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,364,925          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/24/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">724,770          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,732,770          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/23/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">704,815          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,705,399          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/22/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">570,951          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,561,911          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/21/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">530,421          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,499,460          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/20/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">695,330          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,635,312          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/19/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">747,422          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,727,355          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/18/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">646,654          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,396,462          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/17/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">720,378          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,776,960          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/16/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">706,164          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,716,828          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/15/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">589,285          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,522,563          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/14/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">540,268          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,447,177          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/13/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">697,985          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,615,115          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/12/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">754,545          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,669,717          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/11/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">656,284          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,312,178          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/10/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">711,124          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,716,812          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/9/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">709,653          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,608,209          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/8/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">632,498          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,515,902          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/7/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">641,761          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,506,859          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/6/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">755,555          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,748,718          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/5/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">732,123          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,795,369          </td>
              </tr>
      </tbody>
</table>"""

In [11]:
HTMLstring1="""
<table class="views-table views-view-table cols-4 sticky-enabled sticky-table">
        <thead>
      <tr>
                                                  <th class="views-align-center views-field views-field-field-today-date views-align-center" id="view-field-today-date-table-column" scope="col">Date</th>
                                                  <th class="views-align-center views-field views-field-field-2021-throughput views-align-center" id="view-field-2021-throughput-table-column" scope="col">2021 Traveler Throughput </th>
                                                  <th class="views-align-center views-field views-field-field-2020-throughput views-align-center" id="view-field-2020-throughput-table-column" scope="col">2020 Traveler Throughput </th>
                                                  <th class="views-align-center views-field views-field-field-2019-throughput views-align-center" id="view-field-2019-throughput-table-column" scope="col">2019 Traveler Throughput </th>
              </tr>
    </thead>
    <tbody>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/4/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">466,669          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,345,846          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/3/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">718,988          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,184,253          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/2/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">764,761          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,088,760          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">7/1/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">626,516          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,547,889          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/30/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">500,054          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,347,767          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/29/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">625,235          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,455,536          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/28/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">633,810          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,632,030          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/27/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">546,310          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,368,846          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/26/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">632,984          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,730,515          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/25/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">623,624          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,711,222          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/24/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">494,826          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,594,661          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/23/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">471,421          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,506,510          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/22/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">607,540          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,716,428          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/21/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">590,456          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,719,643          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/20/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">507,129          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,378,559          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/19/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">587,908          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,772,903          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/18/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">576,514          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,728,786          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/17/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">441,829          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,552,395          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/16/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">417,924          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,466,574          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/15/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">534,528          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,699,580          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/14/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">544,046          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,642,083          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/13/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">437,119          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,318,946          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/12/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">519,304          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,727,860          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/11/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">502,209          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,675,686          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/10/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">386,969          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,509,058          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/9/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">338,382          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,433,189          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/8/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">430,414          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,644,981          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/7/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">441,255          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,669,860          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/6/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">353,016          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,225,952          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/5/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">419,675          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,649,808          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/4/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">391,882          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,623,947          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/3/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">304,436          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,370,152          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/2/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">267,742          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,247,421          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">6/1/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">353,261          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,499,002          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/31/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">352,947          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,555,578          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/30/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">268,867          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,117,180          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/29/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">327,133          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,570,613          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/28/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">321,776          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,485,770          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/27/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">261,170          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,269,035          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/26/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">264,843          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,453,649          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/25/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">340,769          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,512,237          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/24/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">267,451          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,070,716          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/23/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">253,190          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,124,825          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/22/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">348,673          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,792,670          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/21/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">318,449          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,673,635          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/20/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">230,367          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,472,123          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/19/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">190,477          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,312,727          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/18/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">244,176          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,615,691          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/17/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">253,807          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,620,276          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/16/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">193,340          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,091,116          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/15/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">250,467          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,664,549          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/14/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">234,928          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,611,324          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/13/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">176,667          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,343,675          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/12/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">163,205          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,191,387          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/11/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">215,645          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,512,315          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/10/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">200,815          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,419,114          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/9/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">169,580          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,985,942          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/8/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">215,444          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,602,631          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/7/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">190,863          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,555,342          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/6/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">140,409          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,270,662          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/5/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">130,601          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,106,597          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/4/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">163,692          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,470,969          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/3/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">170,254          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,512,598          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/2/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">134,261          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,968,278          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">5/1/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">171,563          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,546,029          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/30/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">154,695          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,499,461          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/29/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">119,629          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,256,442          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/28/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">110,913          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,102,068          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/27/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">119,854          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,412,770          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/26/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">128,875          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,506,809          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/25/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">114,459          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,990,464          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/24/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">123,464          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,521,897          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/23/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">111,627          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,526,961          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/22/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">98,968          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,254,209          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/21/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">92,859          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,227,475          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/20/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">99,344          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,594,171          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/19/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">105,382          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,356,802          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/18/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">97,236          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,988,205          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/17/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">106,385          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,457,133          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/16/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">95,085          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,616,158          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/15/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">90,784          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,317,381          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/14/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">87,534          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,208,688          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/13/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">102,184          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,484,580          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/12/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">90,510          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,446,801          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/11/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">93,645          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,059,142          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/10/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">108,977          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,590,499          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/9/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">104,090          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,487,398          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/8/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">94,931          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,229,276          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/7/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">97,130          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,091,056          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/6/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">108,310          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,384,091          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/5/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">122,029          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,462,929          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/4/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">118,302          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,011,715          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/3/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">129,763          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,476,884          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/2/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">124,021          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,411,500          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">4/1/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">136,023          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,151,626          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/31/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">146,348          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,026,256          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/30/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">154,080          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,360,053          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/29/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">180,002          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,510,294          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/28/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">184,027          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,172,920          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/27/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">199,644          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,538,384          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/26/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">203,858          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,487,162          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/25/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">239,234          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,273,811          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/24/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">279,018          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,151,913          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/23/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">331,431          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,434,370          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/22/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">454,516          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,542,643          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/21/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">548,132          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,227,181          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/20/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">593,167          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,559,307          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/19/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">620,883          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,513,231          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/18/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">779,631          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,320,885          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/17/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">953,699          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,177,929          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/16/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,257,823          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,465,709          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/15/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,519,192          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,545,742          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/14/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,485,553          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,274,658          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/13/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,714,372          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,634,215          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/12/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,788,456          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,503,924          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/11/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,702,686          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,187,298          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/10/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,617,220          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,122,898          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/9/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,909,363          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,378,673          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/8/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,119,867          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,485,430          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/7/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,844,811          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,156,262          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/6/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,198,517          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,543,689          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/5/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,130,015          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,402,692          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/4/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,877,401          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,143,619          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/3/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">1,736,393          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">1,979,558          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/2/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,089,641          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,257,920          </td>
              </tr>
          <tr>
                                                                                        <td headers="view-field-today-date-table-column" class="views-field views-field-field-today-date views-align-center">3/1/2020          </td>
                                                                                        <td headers="view-field-2021-throughput-table-column" class="views-field views-field-field-2021-throughput views-align-center">          </td>
                                                                                        <td headers="view-field-2020-throughput-table-column" class="views-field views-field-field-2020-throughput views-align-center">2,280,522          </td>
                                                                                        <td headers="view-field-2019-throughput-table-column" class="views-field views-field-field-2019-throughput views-align-center">2,301,439          </td>
              </tr>
      </tbody>
</table>"""

In [76]:
#Read both table
dfs = pd.read_html(HTMLstring)
dfs1=pd.read_html(HTMLstring1)
df=dfs[0]
df1=dfs1[0]
df["Date"]=pd.to_datetime(df["Date"])
df1["Date"]=pd.to_datetime(df1["Date"])
df=df.sort_values("Date")
df1=df1.sort_values("Date")

In [77]:
passengers=pd.DataFrame()
date1=pd.date_range('03-01-2019', '05-31-2020')
passengers["Date"]=date1
for i in range(191):
    passengers.loc[126+i, 'Passengers']=df.loc[199-i,"2019 Traveler Throughput"]
for i in range(11):
    passengers.loc[306+i, 'Passengers']=df.loc[19-i, "2020 Traveler Throughput"]
for i in range(126):
    passengers.loc[i, 'Passengers']=df1.loc[125-i, "2019 Traveler Throughput"]
for i in range(92):
    passengers.loc[366+i, 'Passengers']=df1.loc[125-i, "2020 Traveler Throughput"]
passengers.to_csv(path_or_buf=r'Passengers.csv', index=False)